# Preprocessamento

Load

In [14]:
# imports
import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import rcParams, cm
from scipy import signal
from scipy.signal import stft
from scipy.signal import welch

# filters
def butter_bandpass(data, lowcut, highcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)

# Load dataset
data = np.load('datasets/topicos_cc.npy')
voluntarios, trials, classes, eletrodos, valores = data.shape

print(f'{voluntarios}, {trials}, {classes}, {eletrodos}, {valores}')

10, 3, 8, 4, 1600




**STFT:** O objetivo é definir um step que alcance um resultado de janelas semelhantes entre o domínio do tempo e o domínio da frequência.

**PSD:** Obter os dados no domínio da frequência pelo método Welch, casando também o tamanho da dimensionalidade até o número de janelas.

In [15]:
step = 47
segment = 64

data = data.reshape(voluntarios, trials * classes, eletrodos, valores)

n_win = int((data.shape[-1] - segment) / step) + 1 # 33
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:, :, :, k:(k + segment)] for k in ids]).transpose(1, 2, 3, 0, 4)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
# chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (voluntario, classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (voluntario, classes+ensaios, canais, janelas, linhas)')

Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (10, 24, 4, 33, 64) - (voluntario, classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (10, 24, 4, 33, 51) - (voluntario, classes+ensaios, canais, janelas, linhas)


In [16]:
# PSD Welch 
f, Pxx = welch(data, fs=200, nperseg=64, noverlap=32)

print('Formato (shape) dos dados no domínio da frequência usando o método Welch:')           
print(f'Domínio da frequência: {Pxx.shape} - (voluntário, classes+ensaios, canais, janelas)')

Formato (shape) dos dados no domínio da frequência usando o método Welch:
Domínio da frequência: (10, 24, 4, 33) - (voluntário, classes+ensaios, canais, janelas)
